# Flink

- [Apache Flink® — Stateful Computations over Data Streams](https://flink.apache.org/)

<img src="https://flink.apache.org/img/flink-home-graphic.png" width="800"/>

books:
- Hueske, Fabian / Kalavri, Vasiliki. **Stream Processing with Apache Flink**. 2019. O’Reilly.


actions:
- workbench\DataEngineering\codes\data-engineering-java\streaming\flink

# Architecture

version: LTS 1.20

<img src="https://nightlies.apache.org/flink/flink-docs-release-1.20/fig/processes.svg" width="800"/>

## Flink Program/Client

Client: 准备和发送数据流到JobManager.
- detached mode: 断开与JobManager的连接
- attached mode: 保持连接接收进展报告

## JobManager

JobManager负责协调Flink应用的分布式执行:
- 决定合适调度下一个/下一批任务
- 处理结束的任务或任务执行失败
- 协调checkpoint
- 失败时协调恢复

构成组件:
- ResourceManager: 负责资源分配/回收, 制备Flink集群
  - 例: YARN, Kubernetes, 单独部署. 在单独部署中, ResourceManager只能分布可用的slot, 不能启动新的TaskManager.
  - 负责管理task slot: Flink集群中资源调度的单元
- Dispatcher: 提供了REST接口
  - 提交Flink应用, 并为每个提交的job启动一个新的JobMaster.
  - 同时运行Flink Web UI提供job执行信息.
- JobMaster: 负责管理单个JobGraph的执行.
  - Flink集群中可以同时执行多个job, 每个job有自己的JobMaster.

至少有一个JobManager.
- HA部署中, 一个是leader, 其他是standby.

## TaskManager

TaskManager/worker执行数据流中任务, 缓存和交换数据流.

最少有一个TaskManager.

TaskManager中资源调度的最小单位是task slot.
- TaskManager中task slot的数量: 并发执行任务的数量
- 多个操作符可以在一个task slot中执行: task and operator chain

在分布式执行中, Flink将操作符子任务链成任务task. 每个task由一个线程执行.
- 链接chainning操作符: 优化, 减少线程之间通信和缓存的成本, 增加整体吞吐量的同时减少延迟.

每个TaskManager是一个JVM进程, 使用task slot抽象控制TaskManager可以接受的任务数量.
- 每个task slot表示TaskManager中的一个固定的资源子集: 隔离的内存资源.

slot sharing
- 默认情况下, Flink允许subtask共享slot, 甚至是不同task的subtask, 只要这些subtask属于同一个job.

# API

<img src="https://nightlies.apache.org/flink/flink-docs-release-1.20/fig/levels_of_abstraction.svg" width="500"/>

- stateful and timely stream processing: 在DataStream中以Process Function方式内嵌
  - 处理流中事件, 提供一致的和容错的状态抽象.
  - 注册事件时间/处理时间回调: 支持复杂的计算
- DataStream API: 有界/无界的流
  - 流式风格的API, 提供数据处理的基本构建块: 转换, 联接, 聚合, 窗口, 状态等.
  - 数据类型表示为相应编程语言中的类.
- DataSet API: 有界数据集
  - 提供额外的构建块: 循环, 迭代等.
- Table API
  - 围绕表的声明式DSL, 可以是动态变化的表.
    - 定义所需完成的逻辑操作, 而不是描述如何实现操作.
  - 扩展的关系模型: schema, 操作(select, project, join, group-by, aggregate等)
    - 可以通过用户自定义函数扩展
    - 在执行前通过优化器处理
- SQL: 在语义和表达能力上与Table API相同, 但使用SQL查询表达式.
  - 可以在Table API定义的表上执行

example: 
- https://github.com/apache/flink/tree/master/flink-walkthroughs
- https://github.com/apache/flink/tree/master/flink-examples
- https://github.com/apache/flink-playgrounds

## DataStream API

- `StreamExecutionEnvironment`
- `org.apache.flink.streaming.api`

## Table API & SQL

- `TableEnvironment`: `StreamTableEnvironment`

refs:
- [Table API](https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/dev/table/tableapi/)
- [Data Types](https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/dev/table/types/)
- [SQL](https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/dev/table/sql/overview/)
- [Functions](https://nightlies.apache.org/flink/flink-docs-release-1.20/docs/dev/table/functions/overview/)


## Python API

## Library: CEP

## Library: State Processor API

# Data Structure

## State

例:
- 搜索特定的事件模式: 存储当前遇到的事件序列
- 按分钟/小时/天聚合事件: 存储相应的聚合量
- 在数据点流上训练机器学习模型: 存储模型参数的当前版本
- 需要管理历史性数据: 存储提供高效访问之前出现的事件的数据结构

相关概念:
- 状态容错: checkpoint, savepoint
- 应用的可扩展性: 在多个实例上分布状态
- 状态存储后端

Keyed State: 用内嵌的键/值存储维护
- 在按键/分区数据交换之后, 只能在keyed stream上访问键/值状态
- 按键对齐流和状态确保: 状态更新是局部操作, 不需要事务即可保证一致性

Key Group: 由Keyed State组成
- 是分布Keyed State的原子单元, 与定义的最大并行度数量相同.
- 在执行时, 每个键操作符的实例在一个或多个Key Group中的键上操作.

### State Persistence

Flink使用流重放(stream replay)和检查点(checkpoint)的组合实现容错.
- checkpoint用每个操作符的相应状态, 标记输入流中的特定点.
  - checkpint默认关闭
  - 状态通常使用分布式文件系统存储.
  - 通过分布式快照(distributed snapshot)实现: checkpoint, snapshot同义.
- 通过恢复操作符的状态和重放checkpoint之后的事件记录, 从检查点处恢复流数据流的执行, 同时确保一致性(精确一次的处理语义).
  - 重放时, 数据流的源(source)需要能够恢复到特定的事件记录: 例Apache Kafka.

checkpoint的周期间隔是执行时的容错消耗和恢复时间(需要重放的事件记录数量)的权衡.

checkpoint实现
- [Lightweight Asynchronous Snapshots for Distributed Dataflows](https://arxiv.org/abs/1506.08603)
  - ABS: Asynchronous Barrier Snapshotting. 只持久化无环执行拓扑中操作符状态, 在环数据流中保持最少的记录日志.

> aligned checkpintint

stream barrier: 注入到数据流中, 随数据流中数据一起流动
- 将数据流中记录划分为两部分: 当前快照中的记录, 下一次快照中的记录.
- barrier中待snapshot ID, 表示它之前的记录所属的快照ID.
- barrier不会终端流的流动, 是轻量级的.
- 不同snapshot的barrier可以同时在流中出现: 支持并发的多个快照.
- 事件处理
  - 在哪里输入数据流中?: 在数据流源(source)处, 报告给checkpoint协调者JobManager. 
  - 中间的操作符遇到barrier时, 在所有输出流中注入barrier
  - sink操作符遇到barrier, 给checkpoint协调者确认该snapshot.
  - checkpoint协调者收到所有sink的确认后, 认为snapshot完成.
- 对齐: 接收多个输入流的操作符需要在snapshot barrier上对立输入流
  - 一旦操作符从一个输入流中接收到snapshot barrier n, 不能处理该流中的后续事件记录, 记录在输入缓冲中, 直到收到所有输入流的barrier n.
  - 一旦收到所有输入流的barrier n, 开始输出所有pending的事件, 并注入barrier n.
  - 快照状态, 恢复处理流中记录, 先处理输入缓冲中的记录, 再处理流中记录.
  - 最终, 异步写状态到状态后端.
  - 接收多个输入流的操作符: 所有有多个输入的操作符, shuffle后从多个上游子任务消费其输出流的操作符.
- 执行状态快照
  - 何时执行?: 在接收到输入流的所有snapshot barrier时, 在输出流中注入barrier之前.
  - 存储在状态存储后端: JobManager的内存, 或分布式的可靠存储, 例如HDFS.
  - 快照内容?
    - 每个并行流数据源: 快照开始的偏移量/位置.
    - 每个操作符: 快照中存储的状态的指针.
- 恢复中使用快照
  - 使用最近完成的checkpoint k
  - 重新部署整个分布式数据流: 源从位置Sk处开始读取, 每个操作符设置为checkpoint k中的相应状态.
- savepoint总是对齐的.
  - more: Operations > State & Fault Tolerance > Task Failure Recovery

unaligend ckeckpointing
- 思路: checkpoint中可以包含是操作符状态的in-flight事件记录.
- 事件处理
  - 在源中注入barrier: 避免checkpoint协调者过载
  - 操作符对其输入缓冲中的第一个barrier开始处理, 立即在输出缓冲末尾注入barrier; 操作符标记并异步存储所有涉及的记录, 并创建它的状态的snapshot.
- 适用场景: barrier需要尽快的到达sink, 缓慢移动的数据路径. 
- 恢复
  - 除了在开始处理上游操作符的数据之前恢复in-fligth数据之外, 与对齐的checkpint恢复一致.

批处理程序中的状态:
- 不使用checkpoint, 使用重放这个有界的流执行恢复.
- 使用简化的内存/核外数据结构存储状态, 而不是键值索引.

状态存储后端:
- 内存中hash map
- RocksDB
- checkpoint支持: point-in-time snapshot
  - 由JobManager触发TaskManager的checkpoint执行
  - TaskManager存储状态的snapshot

savepoint: 手动触发的checkpoint

## Time

notions of time:
- processing time: 执行特定操作的机器的系统时间
- event time: 每个独立的事件在产生设备上出现的时间.

水位线watermark
- 参考: [Dataflow Model](https://research.google.com/pubs/archive/43864.pdf)
- 度量事件时间的进展. watermark随数据流流动, 携带了一个时间戳t.
- Watermark(t): 事件时间已经到达时刻t, 带时间戳t'<=t的事件不应该出现.
- 流: 有序流, 乱序流

并行流中的watermark
- 源: 源函数的并行子任务独立的产生各自的watermark.
- 随着watermark在流程序中流动, 推进到达的操作符的事件时间. 当操作符推进事件时间时, 为后继的操作符生成新的watermark.
- 有多个输入流的操作符的当前事件时间是输入流事件时间的最小值.

lateness
- 迟到的元素: 系统的事件时间时钟(由watermark表示)中, 元素的时间戳已过.

windowing
- 数据流上的聚合: 窗口window
  - 例: 最近5分钟的数量(time-driven), 最近100个元素的和(data-driven)
- 窗口分类:
  - 滚动tumbling窗口: 没有重叠
  - 滑动sliding窗口: 有重叠
  - 会话session窗口: 由无活动的间隙分隔

# Connectors

DataStream Connectors
- Kafka

Table API Connectors
- Kafka
- JDBC: MySQL

# Deployment

- Overview
- Java Compatibility
- Resource Providers
- Configuration
- Memory Configuration
- Command-Line Interface
- Elastic Scaling
- Fine-Grained Resource Management
- Speculative Execution
- File Systems
- High Availability
- Metric Reporters
- Trace Reporters
- REPLs
- Security
- Advanced

Deployment modes
- **application mode**: runs on JobManager then exit
  - 例: 在Kubernetes上部署Flink应用
- **per-job mode**: YARN, deprecated in v1.15.
- **session mode**: multiple jobs share one JobManager

# Operations

- State & Fault Tolerance
- Metrics
- Traces
- REST API
- Batch
- Debugging
- Monitoring: [example](https://flink.apache.org/2019/02/21/monitoring-apache-flink-applications-101/)
- Upgrading Applications and Flink Versions
- Production Readiness Checklist

# Internals

## Job and Scheduling

## Task Lifecycle

## File Systems

# Flink CDC
* https://github.com/apache/flink-cdc


Flink CDC is a streaming data integration tool that aims to provide users with a more robust API. It allows users to describe their ETL pipeline logic via YAML elegantly and help users automatically generating customized Flink operators and submitting job. Flink CDC prioritizes optimizing the task submission process and offers enhanced functionalities such as schema evolution, data transformation, full database synchronization and exactly-once semantic.

Deeply integrated with and powered by Apache Flink, Flink CDC provides:

- ✅ End-to-end data integration framework- 
- ✅ API for data integration users to build jobs easily
- ✅ Multi-table support in Source / Sink
- ✅ Synchronization of entire databases
- ✅ Schema evolution capability

actions:
- application-store\data-engineering\flink\cdc: standalone Flink cluster, local submit job(MySQL to Doris)

## Concepts

- data pipeline: source, sink, transform(optional), route(optional), pipeline
- data source
- data sink
- table id: mapping to the storage objects of external system
- transform: dalelte/expand data columns, filter data
- route: source-table, sink-table, repliace-symbol, description
- schema evolution: synchronize upstream schema DDL

## Connectors
* https://nightlies.apache.org/flink/flink-cdc-docs-release-3.3/docs/connectors/pipeline-connectors/overview/

- Apache Doris
- Elasticsearch
- Kafka
- MySQL
- OceanBase
- Paimon
- StarRocks

# Stream Benchmark

- [yahoo/streaming-benchmarks](https://github.com/yahoo/streaming-benchmarks)

Sanket Chintapalli, Derek Dagit, Bobby Evans, et al. "Benchmarking Streaming Computation Engines: Storm, Flink and Spark Streaming. " First Annual Workshop on Emerging Parallel and Distributed Runtime Systems and Middleware. IEEE, 2016.

# More

- flink learning blog. [http://www.54tianzhisheng.cn/](http://www.54tianzhisheng.cn/) 含 Flink 入门、概念、原理、实战、性能调优、源码解析等内容。涉及 Flink Connector、Metrics、Library、DataStream API、Table API & SQL 等内容的学习案例，还有 Flink 落地应用的大型项目案例（PVUV、日志存储、百亿数据实时去…: https://github.com/zhisheng17/flink-learning
- Flink 从0到1学习—— 分享四本 Flink 国外的书和二十多篇 Paper 论文: http://www.54tianzhisheng.cn/2019/06/13/flink-book-paper/
- Ververica Platform: The Enterprise Stream Processing Platform by the Original Creators of Apache Flink®: https://www.ververica.com/platform